In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate,HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage,AIMessage,HumanMessage
from langchain_core.output_parsers import StrOutputParser

def run_with_gpt(query):
    prompt=ChatPromptTemplate.from_messages([
        ('system','''Persona: 당신은 국내 최고의 경마 전문가로서 경마장에서 오래 일한 것은 물론 경마 해설을 비롯한 관련 방송에도 자주 얼굴을 비출만큼 유능하고 유명한 업계 최고의 전문가입니다.

Role: 업계 최고의 전문가로서, 당신은 정확하고 확실한 답변으로 사용자의 답에 충실하게 답할 의무가 있습니다. 당신이 의지해야 할 정보들은 공식 자료들이며, 누군가가 그럴듯하게 정리하거나 조사한 내용에 의지해서는 결코 안됩니다.

Examples:

사용자가 질문을 할때 당신은 최대한 상세하고 자세하게 답변해야 합니다. 예를 들어 경기 일정을 물어보면 당신은 어느 경기장들에서 얼마만큼 경기하는지를 답변해줘야 합니다. 또 경기 일정에 대해 묻는다면, 몇시에 얼마만큼의 말과 기수가 나오는 지 대답해야 합니다. 더욱 상세한 정보를 요구한다면 각 말들과 기수들의 이름까지 말할 수 있을 정도로 자세하고 상세해야 합니다. 또, 사용자가 한 경기장의 일정을 묻는다면 며칠며칠에 있는지도 알려줄 정도로 정확하고 확실한 답변을 해주어야 합니다.
        '''),
        ('user', '{query}'),
    ])

    
    prompt += HumanMessagePromptTemplate.from_template([query])
    model = ChatOpenAI(model='gpt-4o', temperature=0)
    output_parser=StrOutputParser()
    chain = prompt | model | output_parser
    return chain

In [3]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os

vectorstore_path = r'C:\Workspaces\project_ma\vectors_new\vectors_new\vs_pdfs'
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_store = FAISS.load_local(vectorstore_path, embeddings, allow_dangerous_deserialization=True)

retriever = vector_store.as_retriever()
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

def rag_and_prompt(query):
    results = retriever.get_relevant_documents(query)
    # if not results:
    #     return "I couldn't find any relevant information in the context provided. Please refine your query."

    retrieved_data = "\n".join([doc.page_content for doc in results])

    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="""\
        당신은 재치있고 유머러스하고 애교 많은 귀여운 경마 안내 챗봇입니다. 
        질문에 대해 검색된 정보를 바탕으로 아주 상세하고 재미있는 답변을 제공합니다.
        """),
        HumanMessage(content=f"""\
        사용자의 질문에 context만을 이용해 답변해 주세요.
        질문: {query}
        context: {retrieved_data}
        """)
    ])

    output_parser = StrOutputParser()
    chain = prompt | model | output_parser
    return chain.invoke({"query": query})

def ask_question(query):
    response = rag_and_prompt(query)
    return response

In [5]:
query='말 사료는 어떤 것들을 써?'
answer=ask_question(query)
print(answer)

말 사료에 대한 정보는 제공된 컨텍스트에 포함되어 있지 않네요. 하지만 경마에 대한 다양한 정보가 가득하니, 경마에 대해 궁금한 점이 있다면 언제든지 물어보세요! 예를 들어, 경주마의 컨디션을 어떻게 판단하는지, 마권을 어떻게 구매하는지 등에 대해 도움을 드릴 수 있습니다. 🐴✨
